#**Segmentation Automatique des Articles Scientifiques**

##**Introduction**

Ce notebook a pour objectif de segmenter des articles scientifiques en paragraphes distincts afin de préparer les données pour des tâches ultérieures de traitement du langage naturel (NLP). La segmentation est une étape clé pour extraire des informations pertinentes et assurer la qualité des données avant l’analyse.

___________________

In [ ]:
# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Sujet1NLP3D

/content/drive/MyDrive/Sujet1NLP3D


In [ ]:
!pip install tiktoken

###**1. Importation des Bibliothèques Nécessaires**

Charger les bibliothèques qui seront utilisées pour la segmentation et la gestion des données.

In [ ]:
import os
import spacy
import gspread
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import re
from googleapiclient.http import MediaIoBaseDownload
import tiktoken

Ces bibliothèques sont indispensables pour manipuler le contenu des articles, gérer les autorisations Google Drive, et traiter le texte (avec Spacy pour le NLP)

In [ ]:
# Initialisation de Spacy
nlp = spacy.load("en_core_web_sm")

###**2. Authentification avec Google Drive et Google Sheets**

Se connecter à Google Drive et Google Sheets pour accéder aux articles stockés sous forme de fichiers.

In [ ]:
# Authentification avec l'API Google Drive
creds = ServiceAccountCredentials.from_json_keyfile_name('tools/custom-casing-434908-n5-4d3c56433812.json', ["https://www.googleapis.com/auth/drive"])
drive_service = build('drive', 'v3', credentials=creds)

In [ ]:
# Connectez-vous à Google Sheets avec vos informations d'identification
client = gspread.authorize(creds)

In [ ]:
# Ouvrez votre fichier Excel qui contient les articles
sheet = client.open('arxiv_data').sheet1

In [ ]:
# Récupérer les données de l'Excel
data = sheet.get_all_records()

Cette section établit une connexion avec Google Drive pour télécharger les fichiers et Google Sheets pour récupérer les métadonnées des articles.

###**3. Téléchargement et Lecture des Fichiers à Partir de Google Drive**

Télécharger les fichiers texte des articles directement depuis Google Drive pour les traiter localement.

In [ ]:
output_folder  = 'new_segmented_files'

In [ ]:
# Fonction pour extraire l'ID du fichier à partir du lien Google Drive
def extract_file_id(drive_link):
    match = re.search(r'/d/([a-zA-Z0-9_-]+)', drive_link)
    return match.group(1) if match else None

In [ ]:
# Fonction pour télécharger le fichier texte depuis Google Drive
def download_file(file_id, file_name):
    request = drive_service.files().get_media(fileId=file_id)
    file_path = f'/content/{file_name}.txt'
    with open(file_path, 'wb') as f:
        downloader = MediaIoBaseDownload(f, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
    return file_path

Cette fonction permet de télécharger les fichiers PDF/texte contenant les articles, en les sauvegardant localement pour les étapes suivantes de segmentation.

In [ ]:
# Fonction pour compter le nombre de tokens dans un texte
def count_tokens(text, model="gpt-4"):
    encoder = tiktoken.encoding_for_model(model)
    tokens = encoder.encode(text)
    return len(tokens)

In [ ]:
def is_section_digit(text, prev_text, max_section_tokens=12, model="gpt-4", previous_section=None):
    """
    Detects if a line is a section based on the following conditions:
    - The line must contain fewer than max_section_tokens (default 8).
    - The line contains only words (no numbers or symbols).
    - The preceding line (prev_text) must contain a valid section number in sequential order.
    - The section number should not be above 9.9.
    """
    num_tokens = count_tokens(text, model)

    # Condition 1: The token count must be within the limit
    if num_tokens > max_section_tokens:
        return False

    # Condition 2: The current line must contain only words (no digits, symbols)
    digit_match = re.match(r'^([a-zA-Z\s]+)$', text.strip())
    if not digit_match:
        return False

    # Condition 3: The previous line must contain a section number in proper order
    match = re.match(r'^(\d+(\.\d+)*)$', prev_text.strip())
    if not match:
        return False

    current_section = match.group(1)

    # Split the section number into parts
    current_nums = list(map(int, current_section.split('.')))

    # Condition 4: Check that the section number is not above 9.9
    if len(current_nums) > 2 or current_nums[0] > 9 or (len(current_nums) == 2 and current_nums[1] > 9):
        return False

    # Check if the current section is greater than the previous section
    if previous_section is not None:
        previous_nums = list(map(int, previous_section.split('.')))

        if current_nums <= previous_nums:
            return False

        # Ensure the current section is in sequence
        if current_nums[0] > previous_nums[0] + 1:
            return False

    if re.search(r'[^\w\s[.-]]', text):
        return False

    return current_section

In [ ]:
# Fonction pour détecter une section précédée d'un numéro (ex: "4.2. ") et suivie d'une phrase descriptive
def is_section_numbered_title(text, prev_text, max_section_tokens=12, model="gpt-4", previous_section=None):
    """
    Detects if a line is a section based on the format 'X.Y.', where:
    - X is a digit between 1 and 9.
    - Y is a digit between 0 and 9.
    - The line contains fewer than max_section_tokens (default 8).
    - The previous section must be less than or equal to the current section.
    """
    num_tokens = count_tokens(text, model)

    # Condition 1: The token count must be within the limit
    if num_tokens > max_section_tokens:
        return False

    # Condition 2: The current line must start with a valid section number (1.1 to 9.9)
    match = re.match(r'^([1-9])\.(\d)\s', text.strip())
    if not match:
        return False

    current_section = match.group(0).strip()  # Capture the section part (e.g., "4.2")

    # Condition 3: The section must be in proper order compared to the previous one
    if previous_section is not None:
        prev_nums = list(map(int, previous_section.split('.')))
        current_nums = list(map(int, current_section.split('.')))

        # Check if the current section is greater than the previous section
        if current_nums <= prev_nums:
            return False

        if re.search(r'[^\w\s[.-]]', text):
            return False

    return current_section

In [ ]:
# Fonction pour détecter une section simple avec une phrase descriptive sans ponctuation ni chiffres
def is_section_simple_title(text, prev_text, max_section_tokens=12, model="gpt-4"):
    """
    Detects if a line is a section title that:
    - Contains only letters (no numbers or symbols).
    - Is alone on its line.
    - The previous line ends with a period (.).
    """
    num_tokens = count_tokens(text, model)

    # Condition 1: The token count must be within the limit
    if num_tokens > max_section_tokens:
        return False

    # Condition 2: The previous line must end with a period (.)
    if not prev_text.strip().endswith('.'):
        return False

    # Condition 3: The current line must only contain letters and spaces (no digits, no punctuation)
    if not re.match(r'^[a-zA-Z\s]+$', text.strip()):
        return False

    if re.search(r'[^\w\s[.-]]', text):
        return False

    return True

In [ ]:
# Fonction pour détecter si une ligne est une section basée sur des lettres
def is_section_letter(text, prev_text, max_section_tokens=12, model="gpt-4", previous_section=None):
    """
    Detects if a line is a section based on letters (a., b., c., etc.).
    The line must start with:
    - A lowercase letter followed by a period (e.g., 'a. Introduction').
    """
    num_tokens = count_tokens(text, model)

    # Condition 1: The token count must be within the limit
    if num_tokens > max_section_tokens:
        return False

    if re.search(r'[^\w\s[.-]]', text):
        return False

    # Condition 2: Match lowercase letters (a., b., c., etc.)
    letter_match = re.match(r'^([a-e])\.\s', text.strip())

    if letter_match:
        current_letter = letter_match.group(1)

        # Verify alphabetical order
        if previous_section is not None:
            prev_letter = previous_section.lower()
            if ord(current_letter) <= ord(prev_letter):
                return False

        return current_letter

    return False

In [ ]:
# Fonction pour détecter si une ligne est une section avec chiffres romains
def is_section_alpha_romain(text, prev_text, max_section_tokens=12, model="gpt-4", previous_section=None):
    num_tokens = count_tokens(text, model)
    if num_tokens > max_section_tokens:
        return False

    if re.search(r'[^\w\s[.-]]', text):
        return False

    # Modifier pour capturer uniquement la partie avant le point
    roman_match = re.match(r'^([ivxlcdm]+)\.\s', text.strip(), re.IGNORECASE)
    if roman_match:
        current_section = roman_match.group(1).strip().lower()  # Prend seulement la partie avant le point

        roman_to_int = {'i': 1, 'ii': 2, 'iii': 3, 'iv': 4, 'v': 5, 'vi': 6, 'vii': 7, 'viii': 8, 'ix': 9, 'x': 10}
        current_value = roman_to_int.get(current_section)

        # Vérifier que current_value n'est pas None avant la comparaison
        if current_value is not None:
            if previous_section is not None and previous_section in roman_to_int:
                prev_value = roman_to_int[previous_section]
                if current_value <= prev_value:
                    return False
            return current_section  # Retourne la valeur de la section en chiffres romains (ex: 'i', 'ii', etc.)

    return False

In [ ]:
# Fonction pour diviser un paragraphe long en sous-parties
def split_long_paragraph(paragraph, max_tokens=300, token_threshold=250, model="gpt-4"):
    tokens = count_tokens(paragraph, model)
    if tokens <= max_tokens:
        return [paragraph]  # Pas besoin de diviser

    # Diviser le paragraphe d'abord par des points pour obtenir des phrases complètes
    sentences = paragraph.split('.')
    result = []
    current_segment = ""

    # Diviser par des points
    for sentence in sentences:
        if count_tokens(current_segment + sentence, model) < max_tokens:
            current_segment += sentence + '.'
        else:
            # Si le segment actuel est trop long, diviser par d'autres ponctuations
            split_segment = split_by_punctuation(current_segment, max_tokens, token_threshold, model)
            result.extend(split_segment)
            current_segment = sentence + '.'

    # Ajouter le dernier segment
    if current_segment:
        split_segment = split_by_punctuation(current_segment, max_tokens, token_threshold, model)
        result.extend(split_segment)

    return result

In [ ]:
# Fonction pour diviser par des virgules, points-virgules, deux-points, etc.
def split_by_punctuation(text, max_tokens, token_threshold, model):
    """Divise un texte par des virgules et autres ponctuations tout en respectant max_tokens et token_threshold."""
    splitters = [',', ';', ':', '!', '?']
    result = []
    current_segment = ""

    for splitter in splitters:
        if splitter in text:
            sub_segments = text.split(splitter)
            for part in sub_segments:
                if count_tokens(current_segment + part, model) < max_tokens:
                    current_segment += part + splitter
                else:
                    if count_tokens(current_segment, model) >= token_threshold:
                        result.append(current_segment.strip())
                    current_segment = part + splitter

    # Vérifier le dernier segment
    if current_segment and count_tokens(current_segment, model) >= token_threshold:
        result.append(current_segment.strip())

    return result

###**4. Détection et Segmentation des Paragraphes**

Détecter les sections et paragraphes dans chaque article pour les structurer correctement.

In [ ]:
def structure_text(content, token_threshold=250, max_tokens=300, min_tokens=200, model="gpt-4"):
    paragraphs = []
    current_paragraph = []
    lines = content.splitlines()

    prev_line = ""
    previous_section_digit = None
    previous_section_letter = None
    previous_section_alpha_romain = None
    previous_section_numbered_title = None

    section_count = {
        'digit': {},
        'letter': {},
        'alpha_romain': {},
        'numbered_title': {},
        'simple_title': {}
    }

    for line in lines:
        if line.strip():  # Ignore empty lines
            section_digit = is_section_digit(line.strip(), prev_line, max_section_tokens=12,
                                             model=model, previous_section=previous_section_digit)
            section_numbered_title = is_section_numbered_title(line.strip(), prev_line, max_section_tokens=12,
                                                               model=model, previous_section=previous_section_numbered_title)
            section_letter = is_section_letter(line.strip(), prev_line, max_section_tokens=12,
                                               model=model, previous_section=previous_section_letter)
            section_alpha_romain = is_section_alpha_romain(line.strip(), prev_line, max_section_tokens=12,
                                                           model=model, previous_section=previous_section_alpha_romain)
            section_simple_title = is_section_simple_title(line.strip(), prev_line, max_section_tokens=12, model=model)

            # Mise à jour des occurrences des sections
            if section_digit:
                section_count['digit'][section_digit] = section_count['digit'].get(section_digit, 0) + 1
                if section_count['digit'][section_digit] > 1:
                    section_digit = None

            if section_numbered_title:
                section_count['numbered_title'][section_numbered_title] = section_count['numbered_title'].get(section_numbered_title, 0) + 1
                if section_count['numbered_title'][section_numbered_title] > 1:
                    section_numbered_title = None

            if section_letter:
                section_count['letter'][section_letter] = section_count['letter'].get(section_letter, 0) + 1
                if section_count['letter'][section_letter] > 1:
                    section_letter = None

            if section_alpha_romain:
                section_count['alpha_romain'][section_alpha_romain] = section_count['alpha_romain'].get(section_alpha_romain, 0) + 1
                if section_count['alpha_romain'][section_alpha_romain] > 1:
                    section_alpha_romain = None

            if section_simple_title:
                section_count['simple_title'][section_simple_title] = section_count['simple_title'].get(section_simple_title, 0) + 1
                if section_count['simple_title'][section_simple_title] > 1:
                    section_simple_title = None

            if section_digit or section_letter or section_alpha_romain or section_numbered_title or section_simple_title:
                if current_paragraph:
                    paragraph_text = " ".join(current_paragraph).strip()
                    if count_tokens(paragraph_text, model) >= min_tokens:
                        paragraphs.append(paragraph_text)
                    current_paragraph = []

                if section_digit:
                    previous_section_digit = section_digit
                if section_letter:
                    previous_section_letter = section_letter
                if section_alpha_romain:
                    previous_section_alpha_romain = section_alpha_romain
                if section_numbered_title:
                    previous_section_numbered_title = section_numbered_title

                paragraphs.append('\n' + line.upper().strip())
            else:
                current_paragraph.append(line.strip())

            prev_line = line

    if current_paragraph:
        paragraph_text = " ".join(current_paragraph).strip()
        if count_tokens(paragraph_text, model) >= min_tokens:
            paragraphs.append(paragraph_text)

    # 1. Remove paragraphs with mostly digits or punctuation
    filtered_paragraphs = []
    for paragraph in paragraphs:
        tokens = count_tokens(paragraph, model)
        if tokens > 0 and (count_tokens(re.sub(r'[^\d\s]', '', paragraph), model) / tokens) >= 0.4:
            continue
        filtered_paragraphs.append(paragraph)

    paragraphs = filtered_paragraphs

    # 2. Remove section titles with fewer than 5 tokens
    filtered_paragraphs = []
    for paragraph in paragraphs:
        if paragraph.startswith('\n') and count_tokens(paragraph.strip(), model) < 5:
            continue
        filtered_paragraphs.append(paragraph)

    return filtered_paragraphs

'\n    # 1. Remove paragraphs after "references"\n    references_found = False\n    for i, paragraph in enumerate(paragraphs):\n        if \'references\' in paragraph.lower():\n            references_found = True\n            paragraphs = paragraphs[:i]  # Keep only paragraphs before "references"\n            break\n\n    if not references_found:\n        paragraphs = paragraphs[:-10]  # Remove last 3 paragraphs if "references" not found\n'

Cette fonction permet de structurer chaque texte en détectant les sections et paragraphes, en tenant compte du nombre de tokens et de certaines règles de ponctuation.

In [ ]:
# Processus de traitement du texte avec structure par section et division de paragraphes longs
def process_text_file(file_path, title, token_threshold=250, max_tokens=300, min_tokens=200, model="gpt-4"):
    # Lecture complète du fichier
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # Structurer le texte avec sections et paragraphes
    paragraphs = structure_text(content, token_threshold, max_tokens, min_tokens, model)

    # Sauvegarder avec le titre
    formatted_content = f"{title}\n\n"
    for paragraph in paragraphs:
        if is_section_digit(paragraph, "", model=model):  # Ligne seule considérée comme une section
            formatted_content += f"{paragraph}\n"
        else:
            split_paragraphs = split_long_paragraph(paragraph, max_tokens, token_threshold, model)
            for p in split_paragraphs:
                formatted_content += f"{p}\n\n"

    return formatted_content

###**5. Sauvegarde des Fichiers Segmentés**

Enregistrer chaque fichier d'article segmenté dans un nouveau fichier texte structuré.

In [ ]:
# Processus complet de segmentation et filtrage pour chaque fichier
for record in data:
    drive_link = record['Content']  # Extraire le lien de partage
    title = record['Title']  # Récupérer le titre depuis l'Excel
    file_id = extract_file_id(drive_link)

    if file_id:
        file_name = f"article_{data.index(record) + 1}"  # Nommer les fichiers en fonction de leur index
        file_path = download_file(file_id, file_name)

        # Processus de structuration et de sauvegarde des paragraphes
        formatted_content = process_text_file(file_path, title, token_threshold=250, max_tokens=300, min_tokens=200, model="gpt-4")

        # Sauvegarde du fichier structuré et filtré
        output_file = os.path.join(output_folder, f"{file_name}_segmented.txt")
        with open(output_file, 'w', encoding='utf-8') as f_out:
            f_out.write(formatted_content)

        print(f"Fichier segmenté et filtré sauvegardé : {output_file}")
    else:
        print(f"Impossible d'extraire l'ID du fichier pour le lien : {drive_link}")

Fichier segmenté et filtré sauvegardé : new_segmented_files/article_1_segmented.txt
Fichier segmenté et filtré sauvegardé : new_segmented_files/article_2_segmented.txt
Fichier segmenté et filtré sauvegardé : new_segmented_files/article_3_segmented.txt
Fichier segmenté et filtré sauvegardé : new_segmented_files/article_4_segmented.txt
Fichier segmenté et filtré sauvegardé : new_segmented_files/article_5_segmented.txt
Fichier segmenté et filtré sauvegardé : new_segmented_files/article_6_segmented.txt
Fichier segmenté et filtré sauvegardé : new_segmented_files/article_7_segmented.txt
Fichier segmenté et filtré sauvegardé : new_segmented_files/article_8_segmented.txt
Fichier segmenté et filtré sauvegardé : new_segmented_files/article_9_segmented.txt
Fichier segmenté et filtré sauvegardé : new_segmented_files/article_10_segmented.txt
Fichier segmenté et filtré sauvegardé : new_segmented_files/article_11_segmented.txt
Fichier segmenté et filtré sauvegardé : new_segmented_files/article_12_seg

Après la segmentation, chaque fichier est sauvegardé dans un dossier spécifique. Le contenu est structuré et prêt pour une analyse future.

_______________________

##**Conclusion**

Ce notebook permet d’automatiser la segmentation des articles scientifiques en paragraphes distincts, un processus essentiel pour garantir une extraction d’information précise et structurée lors des étapes suivantes du projet.